In [8]:
pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import getpass

os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Langchain's API Key: ")
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI's API Key: ")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = 'https://api.smith.langchain.com'

Langchain's API Key: ········
OpenAI's API Key: ········


In [21]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

#INDEXING

#loading docs
loader = WebBaseLoader(web_path = ("https://lilianweng.github.io/posts/2023-06-23-agent/"),
                       bs_kwargs = dict(
                       parse_only = bs4.SoupStrainer(
                       class_ = ("post-content", "post-title", "post-header"))),)
docs = loader.load()

#split
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
splits = text_splitter.split_documents(docs)

#embed
vectorstore = Chroma.from_documents(documents=splits,
                                   embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

#Retrieval and Generation

prompt = hub.pull("rlm/rag-prompt")

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature = 0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = ({"context" : retriever | format_docs, "question" : RunnablePassthrough() }
            | prompt
            | llm
            | StrOutputParser()
            )
rag_chain.invoke("What are the different types of memory? explain them")


'The different types of memory are Short-Term Memory (STM) or Working Memory, which stores information needed for complex cognitive tasks and has a limited capacity and duration. Long-Term Memory (LTM) can store information for a long time and has two subtypes: Explicit / declarative memory for facts and events, and Implicit / procedural memory for skills and routines.'

In [43]:
#INDEXING - LOADING, SPLITTING, EMBEDDING

question = "What is your favourite superintelligence led company?"
document = "My favourite superintelligence led company is Anthropic"

In [44]:
import tiktoken

def num_tokens_from_string(string : str, encoding_name: str) -> int:
    encoder = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoder.encode(string))
    return num_tokens
num_tokens_from_string(question, "cl100k_base")

9

In [45]:
from langchain_openai import OpenAIEmbeddings
embd = OpenAIEmbeddings()
query_result = embd.embed_query(question)
document_result = embd.embed_query(document)
len(query_result)

1536

In [46]:
#cosine similarity

import numpy as np

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)
similarity = cosine_similarity(query_result, document_result)
similarity

0.9290051854582994

In [56]:
# since the cosine similarity is 0.929, it just means it has high similarity

#loading
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(web_path = ("https://lilianweng.github.io/posts/2023-06-23-agent/"),
                      bs_kwargs = dict(
                      parse_only = bs4.SoupStrainer(
                      class_ = ("post-content", "post-title", "post-header"))),)
blog_docs = loader.load()


In [57]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size = 300, chunk_overlap = 50)
splitter = text_splitter.split_documents(blog_docs)

In [58]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever(search_kwargs={"k": 1})
docs = retriever.get_relevant_documents("What is Task Decomposition?")
len(docs)

1

In [51]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'))])

In [59]:
llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature = 0)
chain = prompt | llm
chain.invoke({"context": docs, "question": "What is Task Decomposition?"})


AIMessage(content='Task Decomposition is a technique that involves breaking down complex tasks into smaller and simpler steps in order to enhance model performance on those tasks.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 182, 'total_tokens': 209}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-12ee699b-34e4-4816-b568-48aa472f3db9-0', usage_metadata={'input_tokens': 182, 'output_tokens': 27, 'total_tokens': 209})

In [60]:
from langchain import hub
prompt_hub_rag = hub.pull("rlm/rag-prompt")

In [61]:
prompt_hub_rag

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [62]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

"Task Decomposition is a technique that involves breaking down a complex task into smaller and simpler steps in order to enhance model performance on complex tasks. This technique transforms big tasks into multiple manageable tasks and sheds light on the interpretation of the model's thinking process."